# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 30 2022 1:40PM,247667,16,SSF_T3_W2,Sartorius Stedim Filters Inc.,Released
1,Sep 30 2022 1:40PM,247667,16,SSF_T4_W2,Sartorius Stedim Filters Inc.,Released
2,Sep 30 2022 1:40PM,247667,16,SSF_T5_W2,Sartorius Stedim Filters Inc.,Released
3,Sep 30 2022 1:20PM,247666,10,CTF0011435,"Citieffe, Inc.",Released
4,Sep 30 2022 1:20PM,247666,10,CTF0011436,"Citieffe, Inc.",Executing
5,Sep 30 2022 1:20PM,247666,10,8593036,"Citieffe, Inc.",Released
6,Sep 30 2022 1:12PM,247664,16,8589399,Sartorius Bioprocess Solutions,Released
7,Sep 30 2022 1:12PM,247664,16,8592471,Sartorius Bioprocess Solutions,Released
8,Sep 30 2022 1:07PM,247661,19,N17788,"NAPP Technologies, LLC",Released
9,Sep 30 2022 12:25PM,247658,16,8593016,Sartorius Stedim North America,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
27,247661,Released,1
28,247664,Released,2
29,247666,Executing,1
30,247666,Released,2
31,247667,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
247658,NaN,NaN,1.0
247661,NaN,NaN,1.0
247664,NaN,NaN,2.0
247666,NaN,1.0,2.0
247667,NaN,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247396,0.0,0.0,1.0
247567,0.0,5.0,13.0
247578,9.0,0.0,2.0
247601,0.0,0.0,6.0
247602,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247396,0,0,1
247567,0,5,13
247578,9,0,2
247601,0,0,6
247602,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247396,0,0,1
1,247567,0,5,13
2,247578,9,0,2
3,247601,0,0,6
4,247602,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247396,,,1
1,247567,,5,13
2,247578,9,,2
3,247601,,,6
4,247602,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 30 2022 1:40PM,247667,16,Sartorius Stedim Filters Inc.
3,Sep 30 2022 1:20PM,247666,10,"Citieffe, Inc."
6,Sep 30 2022 1:12PM,247664,16,Sartorius Bioprocess Solutions
8,Sep 30 2022 1:07PM,247661,19,"NAPP Technologies, LLC"
9,Sep 30 2022 12:25PM,247658,16,Sartorius Stedim North America
10,Sep 30 2022 12:20PM,247656,10,ISDIN Corporation
41,Sep 30 2022 12:16PM,247655,10,Aerogen LTD.
43,Sep 30 2022 12:06PM,247654,16,Sartorius Lab Products and Service
45,Sep 30 2022 11:53AM,247653,10,ISDIN Corporation
46,Sep 30 2022 11:51AM,247645,10,"Citieffe, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 30 2022 1:40PM,247667,16,Sartorius Stedim Filters Inc.,,,3
1,Sep 30 2022 1:20PM,247666,10,"Citieffe, Inc.",,1,2
2,Sep 30 2022 1:12PM,247664,16,Sartorius Bioprocess Solutions,,,2
3,Sep 30 2022 1:07PM,247661,19,"NAPP Technologies, LLC",,,1
4,Sep 30 2022 12:25PM,247658,16,Sartorius Stedim North America,,,1
5,Sep 30 2022 12:20PM,247656,10,ISDIN Corporation,,,31
6,Sep 30 2022 12:16PM,247655,10,Aerogen LTD.,,,2
7,Sep 30 2022 12:06PM,247654,16,Sartorius Lab Products and Service,,,2
8,Sep 30 2022 11:53AM,247653,10,ISDIN Corporation,,,1
9,Sep 30 2022 11:51AM,247645,10,"Citieffe, Inc.",,,8


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 30 2022 1:40PM,247667,16,Sartorius Stedim Filters Inc.,3,,
1,Sep 30 2022 1:20PM,247666,10,"Citieffe, Inc.",2,1,
2,Sep 30 2022 1:12PM,247664,16,Sartorius Bioprocess Solutions,2,,
3,Sep 30 2022 1:07PM,247661,19,"NAPP Technologies, LLC",1,,
4,Sep 30 2022 12:25PM,247658,16,Sartorius Stedim North America,1,,
5,Sep 30 2022 12:20PM,247656,10,ISDIN Corporation,31,,
6,Sep 30 2022 12:16PM,247655,10,Aerogen LTD.,2,,
7,Sep 30 2022 12:06PM,247654,16,Sartorius Lab Products and Service,2,,
8,Sep 30 2022 11:53AM,247653,10,ISDIN Corporation,1,,
9,Sep 30 2022 11:51AM,247645,10,"Citieffe, Inc.",8,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 30 2022 1:40PM,247667,16,Sartorius Stedim Filters Inc.,3,,
1,Sep 30 2022 1:20PM,247666,10,"Citieffe, Inc.",2,1,
2,Sep 30 2022 1:12PM,247664,16,Sartorius Bioprocess Solutions,2,,
3,Sep 30 2022 1:07PM,247661,19,"NAPP Technologies, LLC",1,,
4,Sep 30 2022 12:25PM,247658,16,Sartorius Stedim North America,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 30 2022 1:40PM,247667,16,Sartorius Stedim Filters Inc.,3.0,NaN,NaN
1,Sep 30 2022 1:20PM,247666,10,"Citieffe, Inc.",2.0,1.0,NaN
2,Sep 30 2022 1:12PM,247664,16,Sartorius Bioprocess Solutions,2.0,NaN,NaN
3,Sep 30 2022 1:07PM,247661,19,"NAPP Technologies, LLC",1.0,NaN,NaN
4,Sep 30 2022 12:25PM,247658,16,Sartorius Stedim North America,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 30 2022 1:40PM,247667,16,Sartorius Stedim Filters Inc.,3.0,0.0,0.0
1,Sep 30 2022 1:20PM,247666,10,"Citieffe, Inc.",2.0,1.0,0.0
2,Sep 30 2022 1:12PM,247664,16,Sartorius Bioprocess Solutions,2.0,0.0,0.0
3,Sep 30 2022 1:07PM,247661,19,"NAPP Technologies, LLC",1.0,0.0,0.0
4,Sep 30 2022 12:25PM,247658,16,Sartorius Stedim North America,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2476380,75.0,1.0,9.0
15,990459,35.0,17.0,0.0
16,990643,8.0,0.0,0.0
19,990551,4.0,2.0,0.0
20,495256,1.0,1.0,0.0
21,247396,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2476380,75.0,1.0,9.0
1,15,990459,35.0,17.0,0.0
2,16,990643,8.0,0.0,0.0
3,19,990551,4.0,2.0,0.0
4,20,495256,1.0,1.0,0.0
5,21,247396,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,75.0,1.0,9.0
1,15,35.0,17.0,0.0
2,16,8.0,0.0,0.0
3,19,4.0,2.0,0.0
4,20,1.0,1.0,0.0
5,21,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,75.0
1,15,Released,35.0
2,16,Released,8.0
3,19,Released,4.0
4,20,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21
Status,,,,,,
Completed,9.0,0.0,0.0,0.0,0.0,0.0
Executing,1.0,17.0,0.0,2.0,1.0,0.0
Released,75.0,35.0,8.0,4.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21
0,Completed,9.0,0.0,0.0,0.0,0.0,0.0
1,Executing,1.0,17.0,0.0,2.0,1.0,0.0
2,Released,75.0,35.0,8.0,4.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21
0,Completed,9.0,0.0,0.0,0.0,0.0,0.0
1,Executing,1.0,17.0,0.0,2.0,1.0,0.0
2,Released,75.0,35.0,8.0,4.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()